In [1]:
import numpy as np
import math

vocabulary = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' '] # 27 chars
language = ['e','j','s'] # 3 languages

def read_file(file_name, path = './languageID/'):
    f = open(path+file_name, 'r')
    content = f.read()
    content = content.replace('\n', '')  # take \n as space
    return content

In [2]:
def boc(file):  # get bag of character for input file
    l = len(file)
    voc = dict()
    for i in range(l):
        if file[i] in voc:
            voc[file[i]] += 1
        else:
            voc[file[i]] = 1
    return voc


def count_char(file): # counts of characters
    voc = boc(file)
    count = []
    for v in vocabulary:
        if v in voc:
            count.append(voc[v])
        else:
            count.append(0)
    return count

# hw4.3.1

In [3]:
e_train = []
j_train = []
s_train = []
for i in range(10):
    e_train.append(read_file('e'+str(i)+'.txt'))
    j_train.append(read_file('j'+str(i)+'.txt'))
    s_train.append(read_file('s'+str(i)+'.txt'))
train = {'e':e_train, 'j':j_train, 's':s_train}

In [4]:
#train

In [5]:
p_prior = (10+1/2)/(30+3/2)
log_prior = math.log(p_prior)

print(p_prior)

0.3333333333333333


# hw4.3.2 & 4.3.3

In [6]:
raw_theta = {'e':[0]*27,'j':[0]*27,'s':[0]*27}
for l in language:
    for f in train[l]:
        tmp = count_char(f)
        raw_theta[l] = [raw_theta[l][i] + tmp[i] for i in range(27)]

In [7]:
print(raw_theta['e'])

[910, 168, 325, 332, 1594, 286, 264, 714, 838, 21, 56, 438, 310, 876, 975, 253, 8, 814, 1001, 1212, 403, 140, 234, 17, 209, 9, 2712]


In [8]:
# using log to avoid underflow
log_theta = {'e':[0]*27,'j':[0]*27,'s':[0]*27}
for l in language:
    total = sum(raw_theta[l])
    # using 1/2 smoothing
    log_theta[l] = [math.log((raw_theta[l][i]+1/2))-math.log((total+27/2)) for i in range(27)]

In [9]:
for l in log_theta:
    print(l+":", [round(math.e**log_theta[l][i],4) for i in range(27)])

e: [0.0602, 0.0111, 0.0215, 0.022, 0.1054, 0.0189, 0.0175, 0.0472, 0.0554, 0.0014, 0.0037, 0.029, 0.0205, 0.0579, 0.0645, 0.0168, 0.0006, 0.0538, 0.0662, 0.0801, 0.0267, 0.0093, 0.0155, 0.0012, 0.0138, 0.0006, 0.1792]
j: [0.1318, 0.0109, 0.0055, 0.0172, 0.0602, 0.0039, 0.014, 0.0318, 0.097, 0.0023, 0.0574, 0.0014, 0.0398, 0.0567, 0.0912, 0.0009, 0.0001, 0.0428, 0.0422, 0.057, 0.0706, 0.0002, 0.0197, 0.0, 0.0142, 0.0077, 0.1234]
s: [0.1046, 0.0082, 0.0375, 0.0397, 0.1138, 0.0086, 0.0072, 0.0045, 0.0499, 0.0066, 0.0003, 0.0529, 0.0258, 0.0542, 0.0725, 0.0243, 0.0077, 0.0593, 0.0658, 0.0356, 0.0337, 0.0059, 0.0001, 0.0025, 0.0079, 0.0027, 0.1683]


# hw4.3.4

In [10]:
test = read_file("e10.txt")

e10 = count_char(test)
print(e10)

[164, 32, 53, 57, 311, 55, 51, 140, 140, 3, 6, 85, 64, 139, 182, 53, 3, 141, 186, 225, 65, 31, 47, 4, 38, 2, 498]


# hw4.3.5

In [11]:
# def multinomial(lst):
#     res, i = 1, sum(lst)
#     i0 = lst.index(max(lst))
#     for a in lst[:i0] + lst[i0+1:]:
#         for j in range(1,a+1):
#             res *= i
#             res //= j
#             i -= 1
#     return res
# log_coef=math.log(multinomial(e10))

# log_coef

In [12]:
log_p = {'e':0,'j':0,'s':0}
for l in language:
    tmp_log_theta = [log_theta[l][i]*e10[i] for i in range(27)]
    log_p[l] = sum(tmp_log_theta)
    print(l+": ", round(math.e**(log_p[l]),4))
    # the results are all zero due to the lack of multinomial coef, so still use log form

e:  0.0
j:  0.0
s:  0.0


In [13]:
log_p

{'e': -7841.865447060635, 'j': -8771.433079075032, 's': -8467.282044010557}

# hw4.3.7

In [14]:
def predict(test_f, log_theta, log_prior_p):
    highest_log_p = float("-inf")
    prediction = None
    log_p = {'e':0,'s':0,'j':0}
    for l in language:
        tmp_log_theta = [log_theta[l][i]*test_f[i] for i in range(27)]
        log_p[l] = sum(tmp_log_theta)+log_prior_p[l]
        if log_p[l] > highest_log_p:
            highest_log_p = log_p[l]
            prediction = l
    return prediction

In [15]:
e_test = []
j_test = []
s_test = []
for i in range(10,20):
    e_test.append(read_file('e'+str(i)+'.txt'))
    j_test.append(read_file('j'+str(i)+'.txt'))
    s_test.append(read_file('s'+str(i)+'.txt'))
    
test = {'e':e_test,'j':j_test,'s':s_test}

In [16]:
log_prior_p = {'e':math.log(1/3),'j':math.log(1/3),'s':math.log(1/3)}
e_prediction = []
for f in test['e']:
    test_f = count_char(f)
    e_prediction.append(predict(test_f, log_theta, log_prior_p))
print(e_prediction)    

j_prediction = []
for f in test['j']:
    test_f = count_char(f)
    j_prediction.append(predict(test_f, log_theta, log_prior_p))
print(j_prediction)

s_prediction = []
for f in test['s']:
    test_f = count_char(f)
    s_prediction.append(predict(test_f, log_theta, log_prior_p))
print(s_prediction)
    


['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e']
['j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j']
['s', 's', 's', 's', 's', 's', 's', 's', 's', 's']
